In [0]:
# ---------------------------------------------------------
# 04_Gold_Simulation (Final Fix)
# Goal: Compare "Random Strategy" vs "AI Strategy" to calculate ROI
# ---------------------------------------------------------

from pyspark.ml import PipelineModel
from pyspark.sql.functions import col, sum as _sum, udf, lit
from pyspark.sql.types import FloatType

# 1. Load Silver Data & Saved Pipeline
print("⏳ Loading Silver Data & Saved Pipeline...")
df = spark.table("silver_lending_club")
model_path = "/Volumes/workspace/default/raw_data/gbt_credit_model"
model = PipelineModel.load(model_path)

# --- THE FIX: Convert Text Columns to Numbers (Again) ---
# The model expects Floats, so we must ensure the data matches.
print("🔧 Fixing Data Types for Simulation...")
df_clean = df.select(
    col("default_flag"),
    col("loan_amnt").try_cast("float"),
    col("int_rate").try_cast("float"),
    col("installment").try_cast("float"),
    col("annual_inc").try_cast("float"),
    col("dti").try_cast("float"),
    col("term_clean"),
    col("emp_length_clean"),
    col("home_ownership"),
    col("purpose"),
    col("addr_state")
).dropna()

# 2. Score the Data (Get Probabilities)
print("🔮 Scoring all customers...")
predictions = model.transform(df_clean)

# 3. Extract "Probability of Default"
# The model outputs a vector [Prob_Safe, Prob_Default]. We need the second number.
extract_prob = udf(lambda v: float(v[1]), FloatType())
df_scored = predictions.withColumn("risk_score", extract_prob(col("probability")))

# 4. Simulation Parameters
BUDGET = 50000  # We can only call 50,000 people
COST_PER_CALL = 5.00  # It costs $5 to make a call
RECOVERY_RATE = 0.10  # If we catch a default, we recover 10% of the loan

# ---------------------------------------------------------
# STRATEGY A: RANDOM (The "Old Way")
# ---------------------------------------------------------
print(f"🎲 Simulating Random Strategy (Calling {BUDGET} random people)...")
# Sort by loan_amnt just to randomize the order (or use rand())
random_calls = df_scored.orderBy(col("loan_amnt")) 
random_selection = random_calls.limit(BUDGET)

# Calculate Recovered Amount
random_recovery = random_selection.filter("default_flag = 1") \
    .agg(_sum("loan_amnt")).collect()[0][0] or 0.0

random_roi = (random_recovery * RECOVERY_RATE) - (BUDGET * COST_PER_CALL)

# ---------------------------------------------------------
# STRATEGY B: AI-DRIVEN (The "New Way")
# ---------------------------------------------------------
print(f"🤖 Simulating AI Strategy (Calling top {BUDGET} riskiest)...")
# Sort by Risk Score (Highest first)
ai_calls = df_scored.orderBy(col("risk_score").desc())
ai_selection = ai_calls.limit(BUDGET)

# Calculate Recovered Amount
ai_recovery = ai_selection.filter("default_flag = 1") \
    .agg(_sum("loan_amnt")).collect()[0][0] or 0.0

ai_roi = (ai_recovery * RECOVERY_RATE) - (BUDGET * COST_PER_CALL)

# ---------------------------------------------------------
# 5. THE RESULTS (The "Gold" Table)
# ---------------------------------------------------------
print("-" * 40)
print(f"💰 FINANCIAL IMPACT ANALYSIS")
print("-" * 40)
print(f"Strategy A (Random) Net Profit: ${random_roi:,.2f}")
print(f"Strategy B (AI Model) Net Profit: ${ai_roi:,.2f}")
print(f"🚀 AI LIFT: +${ai_roi - random_roi:,.2f}")
print("-" * 40)

# Create a clean Gold Table for the Dashboard
results = [
    ("Random Selection", float(random_roi)),
    ("AI Selection", float(ai_roi))
]
df_gold = spark.createDataFrame(results, ["Strategy", "Net_Profit"])
display(df_gold)